In [50]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def is_close(self, other, tolerance):
        return abs(self.x - other.x) < tolerance and abs(self.y - other.y) < tolerance

In [51]:
class Edge:
    def __init__(self, p1, p2, x_speed=0, y_speed=0, name="def_edge"):
        self.p1 = p1
        self.p2 = p2
        self.x_speed = x_speed
        self.y_speed = y_speed
        self.name = name

    def is_point_on_edge(self, point):
        return point.x >= min(self.p1.x, self.p2.x) and point.x <= max(self.p1.x, self.p2.x) and point.y >= min(self.p1.y, self.p2.y) and point.y <= max(self.p1.y, self.p2.y)

    def __hash__(self):
        return hash(self.name)
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __ne__(self, other):
        return not self.__eq__(other)

In [52]:
import copy

class Car:
    def __init__(self, name, position, destination, spawn_time, speed = 1):
        self.name = name
        self.position = position
        self.visited_intersection = False
        self.source = copy.deepcopy(position)
        self.destination = destination
        self.speed = speed
        self.spawn_time = spawn_time

    def move(self, x, y):
        self.position.x += x
        self.position.y += y

    def is_at_intersection(self):
        if (self.position.x == 0 and self.position.y > -1 and self.position.y < 1) \
            or (self.position.y == 0 and self.position.x > -1 and self.position.x < 1):
            if not self.visited_intersection:
                self.visited_intersection = True
                if self.destination == 'E':
                    self.position.x = 2
                    self.position.y = 1
                elif self.destination == 'S':
                    self.position.x = 1
                    self.position.y = -1
            return True
        return False

    def distance_to_intersection(self):
        return abs(self.position.x) + abs(self.position.y)

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return self.name == other.name

    def __ne__(self, other):
        return not(self == other)

In [53]:
import pandas as pd

class Controller:
    def __init__(self, cars, edges, speed_reduction_factor=0.5):
        self.cars = cars
        self.edges = edges
        self.time = 0
        self.intersection_block_times = {}
        self.speed_reduction_factor = speed_reduction_factor
        self.edges_with_reduced_speed = []
        self.log_content = []
        self.active_cars = []
        self.car_positions = {}
        self.no_of_cars = len(cars)
        self.cars_passed_intersection = []
        self.edge_car_speeds = {}
        self.log_content.append(f"{self.time} Simulation started with srf {self.speed_reduction_factor}")

    def check_for_collision(self):
        for car1 in self.active_cars:
            for car2 in self.active_cars:
                if car1.name == car2.name:
                    continue
                if car1.position.is_close(car2.position, 1):
                    self.log_content.append(f"{self.time} Collision detected between {car1.name} and {car2.name}")
                    return True

    def predict_collision_at_intersection(self, car):
        car_edge = self.get_edge(car)
        car_x_speed = car_edge.x_speed
        car_y_speed = car_edge.y_speed
        if self.edge_car_speeds.get(car_edge) and self.edge_car_speeds[car_edge].get(car):
            self.log_content.append(f"{self.time} Car {car.name} has reduced speed to {self.edge_car_speeds[car_edge][car][0]}, {self.edge_car_speeds[car_edge][car][1]}")
            car_x_speed = self.edge_car_speeds[car_edge][car][0]
            car_y_speed = self.edge_car_speeds[car_edge][car][1]
        else:
            self.log_content.append(f"{self.time} Car {car.name} is traveling at mps {car_x_speed + car_y_speed}")
        time_to_intersection = self.time + (abs(car.position.x) + abs(car.position.y) ) / max(abs(car_x_speed), abs(car_y_speed))
        self.log_content.append(f"{self.time} car {car.name} time to intersection {time_to_intersection}")
        for intersection_blocking_car in self.intersection_block_times:
            if intersection_blocking_car.name == car.name:
                continue
            if intersection_blocking_car in self.cars_passed_intersection or car in self.cars_passed_intersection:
                self.log_content.append(f"{self.time} car {intersection_blocking_car.name} has passed intersection. ignoring for collision prediction")
                continue
            intersection_block_time = self.intersection_block_times[intersection_blocking_car]
            if intersection_block_time[0] <= time_to_intersection <= intersection_block_time[1]:
                self.log_content.append(f"{self.time} Car {car.name} traveling at speed {car_x_speed + car_y_speed} is blocked by car {intersection_blocking_car.name} from time {intersection_block_time[0]} to {intersection_block_time[1]}")
                return True
        if not self.intersection_block_times.get(car):
            self.intersection_block_times[car] = [time_to_intersection, time_to_intersection + 5]
        return False

    def move_car_and_check_collision(self, car):
        car_edge = self.get_edge(car)
        if car_edge in self.edge_car_speeds.keys() and car in self.edge_car_speeds[car_edge]:
            self.log_content.append(f"{self.time} Car {car.name} is moving at reduced speed {self.edge_car_speeds[car_edge][car][0]}, {self.edge_car_speeds[car_edge][car][1]} while on edge {car_edge.name}")
            car.move(self.edge_car_speeds[car_edge][car][0], self.edge_car_speeds[car_edge][car][1])
        else:
            self.log_content.append(f"{self.time} Car {car.name} is moving at edge's max speed {car_edge.x_speed}, {car_edge.y_speed}")
            car.move(car_edge.x_speed, car_edge.y_speed)
        if self.predict_collision_at_intersection(car):
            self.log_content.append(f"{self.time} Possible collision at intersection for car {car.name}, reducing speed")
            car_edge = self.get_edge(car)
            if car_edge in self.edge_car_speeds.keys() and car in self.edge_car_speeds[car_edge]:
              self.log_content.append(f"{self.time} Car {car.name} already has reduced speed, checking for collision")
              if self.predict_collision_at_intersection(car):
                  self.log_content.append(f"{self.time} Collision possible for car {car.name} after speed reduction")
                  return True
            self.edges_with_reduced_speed.append(car_edge)
            reduced_x_speed = car_edge.x_speed - self.speed_reduction_factor if car_edge.x_speed > 0 else car_edge.x_speed
            reduced_y_speed = car_edge.y_speed - self.speed_reduction_factor if car_edge.y_speed > 0 else car_edge.y_speed
            if car_edge not in self.edge_car_speeds.keys():
                self.edge_car_speeds[car_edge] = {}
            self.edge_car_speeds[car_edge][car] = [reduced_x_speed, reduced_y_speed]
            self.log_content.append(f"{self.time} Speed reduced for car {car.name} to {reduced_x_speed}, {reduced_y_speed}")
            if self.predict_collision_at_intersection(car):
              self.log_content.append(f"{self.time} Collision detected for car {car.name} after reducing speed")
              return True
        return self.check_for_collision()


    def get_edge(self, car):
        for edge in self.edges:
            if edge.is_point_on_edge(car.position):
                # self.log_content.append(f"{self.time} Car {car.name} is on edge {edge.name} while at {car.position.x}, {car.position.y}")
                return edge
        self.log_content.append(f"{self.time} Car {car.name} is not on any edge while at {car.position.x}, {car.position.y}")
        raise Exception(f"Car {car.name} is not on any edge while at ({car.position.x}, {car.position.y})")

    def print_status(self, car):
      car_edge = self.get_edge(car)
      self.log_content.append(f"{self.time} car {car.name} is at {car.position.x}, {car.position.y} on edge {car_edge.name}")
      if not self.car_positions.get(car):
          self.car_positions[car] = []
      self.car_positions[car].append(f'({car.position.x}, {car.position.y})')

    def pad_position_arrays(self):
        largest_array_size = 0
        for car in self.car_positions:
            largest_array_size = max(largest_array_size, len(self.car_positions[car]))
        for car in self.car_positions:
            if car.spawn_time > 0:
                to_pad = [f'({car.source.x}, {car.source.y})'] * car.spawn_time
                to_pad.extend(self.car_positions[car])
                self.car_positions[car] = to_pad
                largest_array_size = max(largest_array_size, len(self.car_positions[car]))
        for car in self.car_positions:
            if len(self.car_positions[car]) < largest_array_size:
                self.car_positions[car].extend([self.car_positions[car][-1]] * (largest_array_size - len(self.car_positions[car])))
                
    def write_log(self, filename, verbose=False):
        self.log_content.append(f"{self.time} Simulation ended with srf {self.speed_reduction_factor}")
        with open(filename, "w") as f:
            for line in self.log_content:
                f.write(line + "\n")
        if verbose:
            print("Log written to", filename)
    
    def write_positions_to_excel(self):
        self.pad_position_arrays()
        df = pd.DataFrame(self.car_positions)
        columns = []
        for i in range(self.no_of_cars):
            columns.append(f"{'SE_' if not i % 2 else 'WS_'}CAR_{i}")
        df.columns = columns
        file_name = "simulation/data/car_positions.xlsx"
        df.to_excel(file_name, index=False)
        print("Positions written to file", file_name)


    def simulate(self):
        while True:
            self.active_cars = [car for car in self.cars if car.spawn_time <= self.time]
            self.log_content.append(f"{self.time} Active cars: {[car.name for car in self.active_cars]}")
            if len(self.active_cars) == 0:
                self.log_content.append(f"{self.time} All cars have left the intersection")
                break
            for car in self.active_cars:
                self.print_status(car)
                if self.move_car_and_check_collision(car):
                    return -1
                car.is_at_intersection()
                if car.visited_intersection:
                    self.cars_passed_intersection.append(car)
                if car.visited_intersection and car.distance_to_intersection() > 10:
                    self.cars.remove(car)
                    self.active_cars.remove(car)
                    self.log_content.append(f"{self.time} {car.name} has left the simulation")
                    if len(self.active_cars) == 0:
                        self.log_content.append(f"{self.time} All cars have left the simulation")
                        return self.time
            self.time += 1

In [54]:
import numpy as np

speed_reduction_factors = np.arange(0, 1.01, 0.01)

class QLearningController(Controller):
    def __init__(self, cars, edges, alpha=0.1, gamma=0.6, epsilon=0.1):
        super().__init__(cars, edges)
        self.q_table = np.zeros((101, 101))  # Q-table for speed_reduction_factor values from 0 to 1
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def update_q_table(self):
      no_of_srfs = len(speed_reduction_factors)
      self.q_table = np.zeros((no_of_srfs, no_of_srfs))
      for i in range(no_of_srfs):
        reward = self.simulate_with_speed_reduction(speed_reduction_factors[i])
        for j in range(no_of_srfs):
            # print("reward", reward, "for", i/100)
            next_max = np.max(self.q_table[i])
            self.q_table[i, j] = (1 - self.alpha) * self.q_table[i, j] + self.alpha * (reward + self.gamma * next_max)


    def simulate_with_speed_reduction(self, speed_reduction_factor):
      if speed_reduction_factor == 1:
        return -1000
      controller_copy = self.copy_controller(speed_reduction_factor)
      controller_copy.speed_reduction_factor = speed_reduction_factor
      try:
        total_time_taken = controller_copy.simulate()
        if total_time_taken == -1:
          return -1000  # Penalize collision heavily
        return -total_time_taken  # Negative total time taken to minimize delay
      except:
        raise Exception("Error in simulation for srf", speed_reduction_factor)
      finally:
        controller_copy.write_log("logs/training_logs/srf_" + str(speed_reduction_factor) + ".log")
      

    def copy_controller(self, speed_reduction_factor):
        cars_copy = [Car(car.name, Point(car.position.x, car.position.y), car.destination, car.spawn_time) for car in self.cars]
        edges_copy = [Edge(edge.p1, edge.p2, edge.x_speed, edge.y_speed, edge.name) for edge in self.edges]
        return Controller(cars_copy, edges_copy, speed_reduction_factor)

# Example usage
edges = [
    Edge(Point(0, -50), Point(0, -40), 0, 1, "e00"),
    Edge(Point(0, -40), Point(0, -30), 0, 1, "e01"),
    Edge(Point(0, -30), Point(0, -20), 0, 1, "e02"),
    Edge(Point(0, -20), Point(0, -10), 0, 1, "e03"),
    Edge(Point(0, -10), Point(0, 40), 0, 1, "e04"),
    Edge(Point(-50, 0), Point(-40, 0), 1, 0, "e10"),
    Edge(Point(-40, 0), Point(-30, 0), 1, 0, "e11"),
    Edge(Point(-30, 0), Point(-20, 0), 1, 0, "e12"),
    Edge(Point(-20, 0), Point(-10, 0), 1, 0, "e13"),
    Edge(Point(-10, 0), Point(40, 0), 1, 0, "e14"),
    Edge(Point(1, -50), Point(1, 40), 0, -1, "e3"),
    Edge(Point(-50, 1), Point(40, 1), 1, 0, "e4"),
]
cars = [
    Car("c1", Point(0, -50), 'E', 0),
    Car("c2", Point(-50, 0), 'S', 0),
    Car("c3", Point(0, -50), 'E', 25),
    Car("c4", Point(-50, 0), 'S', 28),
    Car("c5", Point(0, -50), 'E', 32),
    Car("c6", Point(-50, 0), 'S', 35),
    Car("c7", Point(0, -50), 'E', 38),

    # Car("c11", Point(0, -50), 'E', 45+0),
    # Car("c12", Point(-50, 0), 'S', 45+0),
    # Car("c13", Point(0, -50), 'E', 45+25),
    # Car("c14", Point(-50, 0), 'S', 45+28),
    # Car("c15", Point(0, -50), 'E', 45+32),
    # Car("c16", Point(-50, 0), 'S', 45+35),
    # Car("c17", Point(0, -50), 'E', 45+38),

    # Car("c21", Point(0, -50), 'E', 90+0),
    # Car("c22", Point(-50, 0), 'S', 90+0),
    # Car("c23", Point(0, -50), 'E', 90+25),
    # Car("c24", Point(-50, 0), 'S', 90+28),
    # Car("c25", Point(0, -50), 'E', 90+32),
    # Car("c26", Point(-50, 0), 'S', 90+35),
    # Car("c27", Point(0, -50), 'E', 90+38),
]
# times = [0,0,25,28,32,35,38,45,45,80]
# cars = [Car("c"+str(i), Point(0, -50) if not i%2 else Point(-50,0), 'E' if not i%2 else 'S', times[i]) for i in range(10)]

# cars = [
#     Car("c1", Point(0, -50), 'E', 0),
#     Car("c2", Point(-50, 0), 'S', 0),
# ]
# times = [0, 4]
# times2 = [0, 7]

# cars_east = [Car("a" + str(i), Point(0, -50), 'E', times[i]) for i in range(1)]
# cars_south = [Car("b" + str(i), Point(-50, 0), 'S', times2[i]) for i in range(2)]
# cars = cars_east + cars_south
# print(cars)

controller = QLearningController(cars, edges)
total_time_taken = controller.update_q_table()
q_table = controller.q_table

In [55]:
max_q_values = []
for i in range(len(q_table)):
  max_q_values.append(max(q_table[i]))
most_optimal_srf = speed_reduction_factors[np.argmax(max_q_values)] 
print("most optimal edge weight", 1-most_optimal_srf, "srf", most_optimal_srf)

most optimal edge weight 0.9 srf 0.1


In [56]:
if most_optimal_srf != 0:
    optimal_controller = Controller(cars, edges, most_optimal_srf)
    delay = optimal_controller.simulate()
    optimal_controller.write_log("logs/positions.log")
    optimal_controller.write_positions_to_excel()
else:
    print("No speed reduction needed")

Positions written to file simulation/data/car_positions.xlsx
